# Deep and Convolutional Neural Networks for Image Classification

At surface value there is not much difference between deep neural networks (DNNs) and convolutional neural networks (CNNs). Both are types of artificial neural networks used in machine learning and deep learning. However, they have different architectures and are suited for different types of tasks.

DNNs are composed of multiple layers of interconnected neurons, where each neuron in one layer is connected to every neuron in the next layer. This architecture allows DNNs to learn complex patterns and relationships in data, making them suitable for a wide range of tasks, including image classification, natural language processing, and speech recognition.

CNNs, on the other hand, are specifically designed for processing grid-like data, such as images. They use convolutional layers that apply filters to the input data to extract features, followed by pooling layers that reduce the spatial dimensions of the data. This architecture allows CNNs to effectively capture spatial hierarchies and patterns in images, making them particularly well-suited for image classification tasks.

## Network Architecture Design

Starting with architecture, there are a number of layers that are commonly used in DNNs and CNNs. For DNNs, these include:
- Input Layer: The first layer that receives the input data.
- Hidden Layers: Multiple layers of neurons that process the input data and learn patterns.
- Output Layer: The final layer that produces the output predictions.
For CNNs, the common layers include:
- Convolutional Layers: Apply filters to the input data to extract features.
- Pooling Layers: Reduce the spatial dimensions of the data.
- Fully Connected Layers: Similar to the hidden layers in DNNs, where each neuron is connected to every neuron in the next layer.
- Output Layer: The final layer that produces the output predictions.

We will use tensorflow and keras to build both types of networks for image classification.

In [3]:
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, AveragePooling2D
from tensorflow.keras.optimizers import Adam

In [18]:
#The size of the input images
input_shape = (432,288,3)

#The number of classes
classes_count = 10

In [19]:
model = Sequential([
    Input(input_shape),

    # First convolutional layer
    Conv2D(32,kernel_size=(5,5),activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    # Second convolutional layer
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    # Flatten the feature map
    Flatten(),
    
    # Fully connected layers
    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(64, activation='relu'),
    
    # Output layer
    Dense(classes_count, activation='softmax'),
])

We can print a summary of the model architecture to see the number of layers and parameters in each network.

In [20]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 428, 284, 32)   │         2,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 214, 142, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 212, 140, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 106, 70, 64)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 474880)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │    60,784,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 60,814,602 (231.99 MB)

 Trainable params: 60,814,602 (231.99 MB)

 Non-trainable params: 0 (0.00 B)

## Network Training

In [ ]:
import os
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from datetime import datetime as datetime
from tensorflow.keras.optimizers import Adam

We will need to configure a couple of things before we dive into training the neural network. We need to specify where the training and testing data is located, the number of classes we are trying to predict, the image size, and the batch size. These parameters will be used to configure the data generators as well as the neural network architecture.

In [6]:
BASE_PATH = r"C:\Users\JTWit\Documents\ECE 579\Datasets\Split GTZAN Dataset"
TEST_PATH = os.path.join(BASE_PATH,'test')
TRAIN_PATH = os.path.join(BASE_PATH,'train')

SAVE_PATH = os.path.join(r"C:\Users\JTWit\Documents\ECE 579","Custom DNN Models")

#Make the save path for the neural network just in case it does not yet exist
os.makedirs(SAVE_PATH,exist_ok = True)

checkpoint_dir = os.path.join(r"C:\Users\JTWit\Documents\ECE 579",'Training Checkpoints')
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

# Create the directory if it doesn't exist
os.makedirs(checkpoint_dir, exist_ok=True)

In [3]:
LEARNING_RATE = 1e-5
EPOCHS = 30

TARGET_SIZE = (432,288)

NETWORK_NAME = "GTZAN Custom DNN"

Let's specify some callbacks to help with training. Callbacks are functions that are called during the training process at specific points, such as at the end of an epoch or after a certain number of batches. They can be used to monitor the training process, save the model, and adjust the learning rate.

In [40]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2, 
    patience=2)

earlystop = EarlyStopping(
    monitor='val_acc',
    mode="max", 
    patience=3)

checkpoint = ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True,  
    monitor='val_loss',      
    save_best_only=False,    
    verbose=1                
)


callbacks = [reduce_lr,earlystop] 

With everything in place, we can now compile the models and begin training. We will use the Adam optimizer and categorical cross-entropy loss function for both networks.

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

We will also have to configure the train and validation data generators to load the images from the specified directories, resize them to the desired input shape, and apply any necessary preprocessing.

In [42]:
def get_train_and_validation_data(training_path, training_options, validation_split=0.2):
    # Create an ImageDataGenerator with validation_split
    datagen = ImageDataGenerator(
        rotation_range=training_options["rotation_range"],
        width_shift_range=training_options["width_shift_range"],
        height_shift_range=training_options["height_shift_range"],
        brightness_range=training_options["brightness_range"],
        rescale=1./255,  # Important for scaling pixel values
        validation_split=validation_split
    )

    # Training generator
    train_generator = datagen.flow_from_directory(
        training_path,
        target_size=training_options["target_size"],
        batch_size=training_options["batch_size"],
        class_mode='categorical',
        subset='training'
    )

    # Validation generator
    validation_generator = datagen.flow_from_directory(
        training_path,
        target_size=training_options["target_size"],
        batch_size=training_options["batch_size"],
        class_mode='categorical',
        subset='validation'
    )

    return train_generator, validation_generator

In [ ]:
#Data generators
train_options = {
    "rotation_range": 0,           # Slightly reduced
    "width_shift_range": 0.0,      # Up to 5% shift (1.5 pixels for 30x30)
    "height_shift_range": 0.,     # Up to 5% shift
    "brightness_range": (1, 1), # Gentle brightness adjustment
    "target_size": TARGET_SIZE,
    "batch_size": 16
}

train_gen,valid_gen = get_train_and_validation_data(TRAIN_PATH,train_options)


Found 640 images belonging to 10 classes.
Found 159 images belonging to 10 classes.


In [ ]:
history = model.fit(train_gen, validation_data=valid_gen, epochs=EPOCHS, callbacks =  callbacks)
    
accuracy = history.history['accuracy'][-1]
date_str = datetime.today().strftime('%Y-%m-%d')
name_string = f"{NETWORK_NAME}(accuracy = {accuracy:.4f})(date = {date_str}).keras"

save_file = os.path.join(SAVE_PATH,name_string)

model.save(save_file)

Epoch 1/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.8906 - loss: 0.3229 - val_accuracy: 0.4969 - val_loss: 2.2162 - learning_rate: 0.0010
Epoch 2/30


c:\Users\JTWit\AppData\Local\spyder-6\envs\ECE579\Lib\site-packages\keras\src\callbacks\early_stopping.py:99: UserWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: accuracy,loss,val_accuracy,val_loss,learning_rate
  current = self.get_monitor_value(logs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.9141 - loss: 0.2424 - val_accuracy: 0.4591 - val_loss: 2.3171 - learning_rate: 0.0010
Epoch 3/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 49s 1s/step - accuracy: 0.9312 - loss: 0.2280 - val_accuracy: 0.4340 - val_loss: 2.5203 - learning_rate: 0.0010
Epoch 4/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - accuracy: 0.9359 - loss: 0.1889 - val_accuracy: 0.4717 - val_loss: 2.4472 - learning_rate: 2.0000e-04
Epoch 5/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - accuracy: 0.9609 - loss: 0.1326 - val_accuracy: 0.4780 - val_loss: 2.5027 - learning_rate: 2.0000e-04
Epoch 6/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.9672 - loss: 0.1076 - val_accuracy: 0.4906 - val_loss: 2.5540 - learning_rate: 4.0000e-05
Epoch 7/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.9594 - loss: 0.1190 - val_accuracy: 0.5094 - val_loss: 2.6274 - learning_rate: 4.0000e-05
Epoch 8/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - accuracy: 0.9422 - loss: 0.1615 - val_accur

## Network Evaluation

In [9]:
from tensorflow.keras.models import load_model

With a trained neural network in hand, we can evaluate its performance on the test dataset. This involves using the model to make predictions on the test data and comparing those predictions to the true labels to calculate metrics such as accuracy, precision, recall, and F1-score.

In [10]:
model_path = r"C:\Users\JTWit\Documents\ECE 579\Custom DNN Models\GTZAN Custom DNN(accuracy = 0.9625)(date = 2026-02-04).keras"
model = load_model(model_path)

In [1]:
def get_test_data(test_path, testing_options):
    test_datagen = ImageDataGenerator(
        rotation_range=testing_options["rotation_range"],
        width_shift_range=testing_options["width_shift_range"],
        height_shift_range=testing_options["height_shift_range"],
        brightness_range=testing_options["brightness_range"],
        rescale=1./255
    )
    test_generator = test_datagen.flow_from_directory(
        test_path,
        target_size=testing_options["target_size"],
        batch_size=testing_options["batch_size"],
        class_mode='categorical',
    )
    return test_generator

In [ ]:
test_options = {
    "rotation_range": 0,              
    "width_shift_range": 0,
    "height_shift_range": 0,
    "brightness_range": (1, 1),       
    "target_size": TARGET_SIZE,
    "batch_size": 16
}

test_gen = get_test_data(TEST_PATH,test_options)

results = model.evaluate(test_gen)
print('Test loss:', results[0])
print('Test accuracy:', results[1])

Found 200 images belonging to 10 classes.
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 326ms/step - accuracy: 0.4700 - loss: 2.4680
Test loss: 2.467993974685669
Test accuracy: 0.4699999988079071


## Object Oriented Implementation

In [ ]:
import os
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from datetime import datetime as datetime
from tensorflow.keras.optimizers import Adam

from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input, AveragePooling2D

from tensorflow.keras.models import load_model


import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt

class CnnTrainer:

    #=================================================
    # Initialization Method
    #=================================================
    def __init__(self,BASE_PATH:str, TEST_DIR:str = 'Test', TRAIN_DIR:str = 'Train',SAVE_DIR:str = 'Save', INPUT_SHAPE:tuple = (432,288,3)):
        
        #Make Dirs
        self.BASE_PATH = BASE_PATH
        self.TRAIN_PATH = os.path.join(self.BASE_PATH,TRAIN_DIR)
        self.TEST_PATH = os.path.join(self.BASE_PATH,TEST_DIR)
        self.SAVE_PATH = os.path.join(self.BASE_PATH,SAVE_DIR)

        #Make sure the save dir exists
        os.makedirs(SAVE_PATH,exist_ok=True)

        #Configure Variables
        self.model = None

        self.INPUT_SIZE = INPUT_SHAPE
        self.TARGET_SIZE = (INPUT_SHAPE[0],INPUT_SHAPE[1])


    #=================================================
    # Model Generation Method
    #=================================================
    def __get_default_model(self):
        self.model = Sequential([
                            Input(self.INPUT_SIZE),

                            # First convolutional layer
                            Conv2D(32,kernel_size=(5,5),activation='relu'),
                            MaxPooling2D(pool_size=(2, 2)),
                            
                            # Second convolutional layer
                            Conv2D(64, kernel_size=(3, 3), activation='relu'),
                            MaxPooling2D(pool_size=(2, 2)),
                            
                            # Flatten the feature map
                            Flatten(),
                            
                            # Fully connected layers
                            Dense(128, activation='relu'),
                            Dropout(0.5),

                            Dense(64, activation='relu'),
                            
                            # Output layer
                            Dense(classes_count, activation='softmax'),
                        ])

    def load_model(self,model):
        self.model = model

    def load_model_from_path(self,model_path:str):
        model = load_model(model_path)


    #=================================================
    # Data Preparation
    #=================================================
    def __get_train_and_validation_data(self, training_options, validation_split=0.2):
        # Create an ImageDataGenerator with validation_split
        datagen = ImageDataGenerator(
            rotation_range=training_options["rotation_range"],
            width_shift_range=training_options["width_shift_range"],
            height_shift_range=training_options["height_shift_range"],
            brightness_range=training_options["brightness_range"],
            rescale=1./255,  # Important for scaling pixel values
            validation_split=validation_split
        )

        # Training generator
        train_generator = datagen.flow_from_directory(
            self.TRAIN_PATH,
            target_size=training_options["target_size"],
            batch_size=training_options["batch_size"],
            class_mode='categorical',
            subset='training'
        )

        # Validation generator
        validation_generator = datagen.flow_from_directory(
            self.TRAIN_PATH,
            target_size=training_options["target_size"],
            batch_size=training_options["batch_size"],
            class_mode='categorical',
            subset='validation'
        )

        return train_generator, validation_generator
    

    def __get_test_data(self, testing_options):
        test_datagen = ImageDataGenerator(
            rotation_range=testing_options["rotation_range"],
            width_shift_range=testing_options["width_shift_range"],
            height_shift_range=testing_options["height_shift_range"],
            brightness_range=testing_options["brightness_range"],
            rescale=1./255
        )
        test_generator = test_datagen.flow_from_directory(
            TEST_PATH,
            target_size=testing_options["target_size"],
            batch_size=testing_options["batch_size"],
            class_mode='categorical',
        )
        return test_generator

    #=================================================
    # Training 
    #=================================================
    def train(self,model_name:str, EPOCHS:int = 50,LEARNING_RATE:float = 1e-3, VAL_SPLIT:float = 0.2, callbacks:list = [], train_options:dict = {}):

        #Check to see if the model has been defined for this instance of the object
        if not self.model:
            warnings.warn("Custom model has not been loaded. Defaulting to predefined model.", UserWarning)
            self.__get_default_model()
        
        #Compile the model
        model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])
        
        #Prepare the training data
        if not train_options:
            train_options = {
                "rotation_range": 0,           
                "width_shift_range": 0.0,      
                "height_shift_range": 0.,    
                "brightness_range": (1, 1), 
                "target_size": self.TARGET_SIZE,
                "batch_size": 16
            }

        train_gen,valid_gen = get_train_and_validation_data(self.TRAIN_PATH,train_options)        

        #Configure default callbacks if none are specified
        if not callbacks:
            reduce_lr = ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,      
                patience=5,      
                min_lr=1e-6,     
                verbose=1
            )

            earlystop = EarlyStopping(
                monitor='val_accuracy',
                mode="max", 
                patience=10,     
                restore_best_weights=True 
            )

            checkpoint = ModelCheckpoint(
                filepath=checkpoint_prefix,
                save_weights_only=True,  
                monitor='val_loss',      
                save_best_only=False,    
                verbose=1                
            )


            callbacks = [reduce_lr,earlystop] 

        #Train the neural network model
        history = model.fit(train_gen, validation_data=valid_gen, epochs=EPOCHS, callbacks =  callbacks)

        #Keep track of the training and acccuracy of the model and create the name string
        accuracy = history.history['accuracy'][-1]
        date_str = dt.today().strftime('%Y-%m-%d')
        name_string = f"{model_name}(accuracy = {accuracy:.4f})(date = {date_str}).keras"

        save_file = os.path.join(self.SAVE_PATH,name_string)

        # Save the model
        model.save(save_file)


    #=================================================
    # Evaluating
    #=================================================
    def evaluate(self, test_options:dict = {}):

        if not test_options:
            test_options = {
                "rotation_range": 0,              
                "width_shift_range": 0,
                "height_shift_range": 0,
                "brightness_range": (1, 1),       
                "target_size": self.TARGET_SIZE,
                "batch_size": 16
            }

        test_gen = get_test_data(self.TEST_PATH,test_options)

        results = model.evaluate(test_gen)
        print('Test loss:', results[0])
        print('Test accuracy:', results[1])
    

    def confusion_matrix(self):
   
        test_options = {
            "target_size": self.TARGET_SIZE,
            "batch_size": 16,
            "shuffle": False # Crucial for alignment
        }
        test_gen = get_test_data(self.TEST_PATH, test_options)

      
        print("Generating predictions...")
        preds = self.model.predict(test_gen)
        y_pred = np.argmax(preds, axis=1)
        y_true = test_gen.classes
        class_labels = list(test_gen.class_indices.keys())

        cm = confusion_matrix(y_true, y_pred)

        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=class_labels, yticklabels=class_labels)
        plt.title('Confusion Matrix')
        plt.ylabel('Actual Labels')
        plt.xlabel('Predicted Labels')
        plt.show()

        # 5. Print Classification Report
        print("\nClassification Report:\n")
        print(classification_report(y_true, y_pred, target_names=class_labels))